In [107]:
import numpy as np
import pandas as pd
import re
import string
import torch
import matplotlib.pyplot as plt
import os
import torchutils as tu
import json

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchmetrics import Accuracy
from torchmetrics.classification import MulticlassConfusionMatrix, MulticlassAveragePrecision
from torchmetrics.classification import MulticlassPrecision, MulticlassRecall, MulticlassF1Score
from time import time
from collections import Counter
from typing import Union
from dataclasses import dataclass
import torch.nn.functional as F

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
stop_words = set(stopwords.words('russian'))
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /home/lbeno/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [108]:
import sys
sys.path.append('..')
from models.lamberts_funcs import data_preprocessing
from models.lamberts_funcs import my_padding, train3, logs_dict_multi_metrics

In [109]:
#from transformers import BertTokenizer

#tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

In [110]:
mystem = Mystem()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [111]:
file_path = '../data/kinopoisk.jsonl'
df = pd.read_json(file_path, lines=True)
df = df.drop_duplicates(subset=['content'])
df_sampled = df[df['grade3'] == 'Good'].sample(frac=0.825, random_state=42)
df_remaining = df.drop(df_sampled.index)
df_train = df_remaining[['content', 'grade3']]
df

,part,movie_name,review_id,author,date,title,grade3,grade10,content
0,top250,Блеф (1976),17144,Come Back,2011-09-24,Плакали наши денежки ©,Good,10.0,"\n""Блеф» — одна из моих самых любимых комедий...."
1,top250,Блеф (1976),17139,Stasiki,2008-03-04,None,Good,0.0,\nАдриано Челентано продолжает радовать нас св...
2,top250,Блеф (1976),17137,Flashman,2007-03-04,None,Good,10.0,"\nНесомненно, это один из великих фильмов 80-х..."
3,top250,Блеф (1976),17135,Sergio Tishin,2009-08-17,""" Черное, красное, ерунда это все. Выигрывает ...",Good,0.0,\nЭта фраза на мой взгляд отражает сюжет несом...
4,top250,Блеф (1976),17151,Фюльгья,2009-08-20,"«Он хотел убежать? Да! Блеф, блеф…»",Neutral,7.0,"\n- как пела Земфира, скорее всего, по соверше..."
...,...,...,...,...,...,...,...,...,...
36586,bottom100,Цветок дьявола (2010),25123,bestiya163,2010-09-23,"Ой, ой, ой!",Bad,2.0,\n Ну с чего бы начать… Давненько я не пи...
36587,bottom100,Цветок дьявола (2010),25192,Молка,2010-10-02,Молчаливый мужик на коне…,Bad,1.0,"\n Можно начать с того, что уже постер к ..."
36588,bottom100,Цветок дьявола (2010),25080,jetry,2010-09-16,Это проявилось сегодня ночью.,Good,7.0,"\n Фильм производства России, поэтому мно..."
36589,bottom100,Цветок дьявола (2010),25088,Alkort,2010-09-16,«Finita la comedia»,Bad,0.0,\n 16 сентября на большие экраны вышел «м...


In [112]:
df_train['grade3'].value_counts()

grade3
Good       4769
Bad        4751
Neutral    4575
Name: count, dtype: int64

In [113]:
ordinal_encoder = OrdinalEncoder(categories=[['Bad', 'Neutral', 'Good']])
df_train['grade3'] = ordinal_encoder.fit_transform(df_train[['grade3']])

In [114]:
df_train['content'] = df_train['content'].apply(data_preprocessing)
df_train = df_train[df_train['content'].str.strip().astype(bool)]


In [115]:
#df['content'] = df['content'].apply(data_preprocessing)
#df = df[df['content'].str.strip().astype(bool)]

In [116]:
#corpus = [word for text in df['content'] for word in text.split()]
#count_words = Counter(corpus)

#sorted_words = count_words.most_common()

In [117]:
#sorted_words = get_words_by_freq(sorted_words)

In [118]:
#vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

In [119]:
with open("../models/vocab_to_int.json", "r", encoding="utf-8") as json_file:
    vocab_to_int = json.load(json_file)

In [120]:
reviews_int = []
for text in df_train['content']:

    r = [vocab_to_int[word] for word in text.split() if vocab_to_int.get(word)]
    reviews_int.append(r)

In [121]:
review_len = [len(x) for x in reviews_int]
df_train['Review len'] = review_len
df_train.head()

,content,grade3,Review len
4,петь земфира скоро совершенно повод фильм челе...,1.0,185
7,финансовый акула белль дюк иметь давний счеты ...,1.0,139
11,действие фильм разворачиваться примерно е год ...,2.0,119
13,должный поставлять против весь восхищаться воо...,2.0,128
15,вор вор дубинка украсть эпиграф красный нить п...,2.0,176


In [122]:
print(f"Средняя длина отзыва: {df_train['Review len'].mean():.2f} слов")
print(f"Медианная длина отзыва: {df_train['Review len'].median()} слов")
print(f"Максимальная длина отзыва: {df_train['Review len'].max()} слов")
print(f"Минимальная длина отзыва: {df_train['Review len'].min()} слов")

Средняя длина отзыва: 174.46 слов
Медианная длина отзыва: 145.0 слов
Максимальная длина отзыва: 1660 слов
Минимальная длина отзыва: 3 слов


In [123]:
SEQ_LEN = 111
features = my_padding(reviews_int, SEQ_LEN, padding='right', truncating='right')

In [124]:
X_train, X_valid, y_train, y_valid = train_test_split(
    features,
    df_train['grade3'].to_numpy(),
    test_size=0.2,
    random_state=42,
    stratify=df_train['grade3'].to_numpy()
)

In [125]:
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
valid_data = TensorDataset(torch.from_numpy(X_valid), torch.from_numpy(y_valid))

BATCH_SIZE = 128

train_loader = DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)

VOCAB_SIZE = len(vocab_to_int)+1 

In [126]:
DEVICE = 'cuda'

In [127]:
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)

In [128]:
@dataclass
class ConfigRNN:
    vocab_size: int
    device: str
    n_layers: int
    embedding_dim: int
    hidden_size: int
    seq_len: int
    bidirectional: Union[bool, int]

In [129]:
net_config = ConfigRNN(
    vocab_size=len(vocab_to_int) + 1,
    device=DEVICE,
    n_layers=4,
    embedding_dim=32,
    hidden_size=32,
    seq_len=SEQ_LEN,
    bidirectional=False,
)

In [130]:
class LSTMClassifier(nn.Module):
    def __init__(self, rnn_conf=net_config) -> None:
        super().__init__()

        self.embedding_dim = rnn_conf.embedding_dim
        self.hidden_size = rnn_conf.hidden_size
        self.bidirectional = rnn_conf.bidirectional
        self.n_layers = rnn_conf.n_layers

        self.embedding = nn.Embedding(rnn_conf.vocab_size, self.embedding_dim)
        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.hidden_size,
            bidirectional=self.bidirectional,
            batch_first=True,
            num_layers=self.n_layers,
        )
        self.bidirect_factor = 2 if self.bidirectional else 1
        self.attention = nn.Linear(self.hidden_size * self.bidirect_factor, 1)
        self.clf = nn.Sequential(
            nn.Linear(self.hidden_size * self.bidirect_factor, 32),
            nn.Tanh(),
            nn.Dropout(),
            nn.Linear(32, 3)  # Три класса
        )

    def model_description(self):
        direction = "bidirect" if self.bidirectional else "onedirect"
        return f"lstm_{direction}_{self.n_layers}"

    def forward(self, x: torch.Tensor):
        embeddings = self.embedding(x)
        out, _ = self.lstm(embeddings)
        attn_weights = torch.softmax(self.attention(out), dim=1)  # (batch_size, seq_len, 1)
        context = torch.sum(out * attn_weights, dim=1)  # (batch_size, hidden_size * num_directions)
        out = self.clf(context)
        return out
    
net_config.bidirectional = True
model_lstm = LSTMClassifier(net_config)
model_lstm.to(device)
tu.get_model_summary(model_lstm, sample_x.to(device))

Layer               Kernel          Output       Params            FLOPs
0_embedding       [32, 21838]   [128, 111, 32]   698,816          14,208
1_lstm                      -   [128, 111, 64]    92,160   8,009,220,096
2_attention           [64, 1]    [128, 111, 1]        65       1,804,416
3_clf.Linear_0       [64, 32]        [128, 32]     2,080         520,192
4_clf.Tanh_1                -        [128, 32]         0          20,480
5_clf.Dropout_2             -        [128, 32]         0               0
6_clf.Linear_3        [32, 3]         [128, 3]        99          24,192
Total params: 793,220
Trainable params: 793,220
Non-trainable params: 0
Total FLOPs: 8,011,603,584 / 8.01 GFLOPs
------------------------------------------------------------------------
Input size (MB): 0.11
Forward/backward pass size (MB): 10.61
Params size (MB): 3.03
Estimated Total Size (MB): 13.75


In [131]:
criterion = nn.CrossEntropyLoss()
optimizer_lstm = torch.optim.Adam(model_lstm.parameters(), lr=0.0005)

In [132]:
metrics = {
    'Accuracy': Accuracy(task='multiclass', num_classes=3).to(device),
    'precision_metric': MulticlassPrecision(num_classes=3, average='macro').to(device),
    'recall_metric': MulticlassRecall(num_classes=3, average='macro').to(device),
    'f1_metric': MulticlassF1Score(num_classes=3, average='macro').to(device),
    'averagepre_metric': MulticlassAveragePrecision(num_classes=3, average='macro').to(device)
}

In [133]:
train_losses, val_losses, train_metrics, val_metrics, rnn_time, predicts = train3(
    epochs=100,
    model=model_lstm,
    train_loader=train_loader,
    valid_loader=valid_loader,
    optimizer=optimizer_lstm,
    criterion=criterion,
    metrics=metrics,
    rnn_conf=net_config,
    patience=15,
    save_path=f'../models/best_{type(model_lstm).__name__}_weights.pth',
    attention=True,
    multiclass=True
)

Epoch 1
train_loss : 1.0941 val_loss : 1.0881
Accuracy - train: 0.3581, val: 0.3679
precision_metric - train: 0.3584, val: 0.2548
recall_metric - train: 0.3581, val: 0.3665
f1_metric - train: 0.3581, val: 0.2831
averagepre_metric - train: 0.3628, val: 0.4123
Сохранены лучшие веса модели после 1 эпохи с валид. лоссом: 1.0881
Epoch 2
train_loss : 1.0689 val_loss : 0.9929
Accuracy - train: 0.4192, val: 0.5089
precision_metric - train: 0.4198, val: 0.5009
recall_metric - train: 0.4178, val: 0.5067
f1_metric - train: 0.4144, val: 0.5006
averagepre_metric - train: 0.4279, val: 0.5204
Сохранены лучшие веса модели после 2 эпохи с валид. лоссом: 0.9929
Epoch 3
train_loss : 0.9665 val_loss : 0.9499
Accuracy - train: 0.5301, val: 0.5366
precision_metric - train: 0.5194, val: 0.5347
recall_metric - train: 0.5270, val: 0.5328
f1_metric - train: 0.5136, val: 0.5045
averagepre_metric - train: 0.5364, val: 0.5603
Сохранены лучшие веса модели после 3 эпохи с валид. лоссом: 0.9499
Epoch 4
train_loss : 0

In [134]:
logs = train_losses, val_losses, train_metrics, val_metrics
df_metrics = logs_dict_multi_metrics(logs)
val_losses = df_metrics.loc['val_loss']
sorted_val_losses = val_losses.sort_values()
top_20_epochs = sorted_val_losses.index[:20]
df_top_20 = df_metrics[top_20_epochs]
df_top_20

,Epoch 7,Epoch 9,Epoch 10,Epoch 8,Epoch 6,Epoch 5,Epoch 11,Epoch 4,Epoch 12,Epoch 14,Epoch 3,Epoch 13,Epoch 2,Epoch 15,Epoch 16,Epoch 17,Epoch 18,Epoch 1,Epoch 19,Epoch 20
train_loss,0.80,0.73,0.71,0.76,0.83,0.86,0.67,0.91,0.64,0.59,0.97,0.61,1.07,0.55,0.52,0.48,0.47,1.09,0.43,0.40
val_loss,0.85,0.86,0.86,0.86,0.86,0.88,0.89,0.90,0.93,0.94,0.95,0.95,0.99,1.01,1.03,1.07,1.08,1.09,1.15,1.20
train_Accuracy,0.64,0.68,0.69,0.66,0.63,0.60,0.71,0.57,0.74,0.76,0.53,0.75,0.42,0.78,0.80,0.82,0.82,0.36,0.85,0.86
train_precision_metric,0.63,0.67,0.68,0.65,0.62,0.59,0.71,0.56,0.73,0.76,0.52,0.75,0.42,0.78,0.80,0.82,0.82,0.36,0.85,0.86
train_recall_metric,0.64,0.68,0.69,0.66,0.63,0.60,0.71,0.57,0.73,0.76,0.53,0.75,0.42,0.78,0.80,0.82,0.82,0.36,0.85,0.86
train_f1_metric,0.63,0.67,0.68,0.65,0.62,0.59,0.71,0.55,0.73,0.76,0.51,0.75,0.41,0.78,0.80,0.82,0.82,0.36,0.85,0.86
train_averagepre_metric,0.67,0.71,0.73,0.69,0.64,0.62,0.75,0.58,0.77,0.80,0.54,0.79,0.43,0.82,0.84,0.86,0.86,0.36,0.89,0.90
val_Accuracy,0.60,0.60,0.61,0.61,0.59,0.58,0.60,0.57,0.59,0.60,0.54,0.60,0.51,0.59,0.58,0.58,0.58,0.37,0.58,0.59
val_precision_metric,0.58,0.59,0.60,0.60,0.58,0.57,0.59,0.56,0.59,0.59,0.53,0.59,0.50,0.58,0.57,0.59,0.58,0.25,0.59,0.58
val_recall_metric,0.60,0.60,0.61,0.60,0.59,0.58,0.59,0.57,0.59,0.60,0.53,0.60,0.51,0.58,0.58,0.58,0.58,0.37,0.58,0.58


In [135]:
if 'фильм' in vocab_to_int:
    print(vocab_to_int['фильм'])

1


In [136]:
speech = df['content'][3]

In [137]:
print(speech)


Эта фраза на мой взгляд отражает сюжет несомненно прекрасного фильма. Есть в ней один человек — Феликс, молодой, экстравагантный, хитрый и, одновременно бывающий очень серьезным. Нужно ли кому-то говорить что в рулетке только одно зеро… А есть все остальные, разделенные по сути на две команды. Они также очень находчивы, но в итоге оказывается что чего-то им не хватает, может быть простоты, может этой самой находчивости. 

Разумеется фильм далек от философии, или чего бы то ни было высокого. Фильм для того чтобы повеселиться, и его главная задача — смешить зрителей на протяжении всего фильма — выполнена на 5+. Не буду говорить о талантливых артистах, один лишь список актеров позволит вам понять что с ними в этом фильме все хорошо. 

Хотелось лишь сказать о самых больших плюсах фильма. Это естественно харизматичные актеры, Энтони Куинн и Челентано просто идеально вписались в эти роли. Великолепные шутки, очень ненавязчивые, что оставляет лишь приятное впечатление. Во всяком случае то, ч

In [138]:
type(reviews_int)

list

In [139]:
# Сохранение всей модели
torch.save(model_lstm, 'model_lstm.pth')


In [140]:
def l_predict_class(model, text, preprocess_fn, padding, seq_len, vocabulary, device='cuda'):
    model.to(device)
    model.eval()  # Устанавливаем модель в режим оценки
    
    # Предобработка текста
    text = preprocess_fn(text)
    text_list = [vocabulary[word] for word in text if word in vocabulary]

    # Пэддинг и преобразование в тензор
    padded_text = padding([text_list], seq_len, padding='right', truncating='right')[0]
    text_tensor = torch.tensor(padded_text, dtype=torch.long).to(device).unsqueeze(0)

    # Прогон через модель
    with torch.no_grad():  # Отключаем градиенты
        output = model(text_tensor)  # Получаем только один выходной тензор
    
    # Преобразуем логиты в вероятности
    probabilities = F.softmax(output, dim=1)
    
    # Получаем предсказанный класс и вероятность
    predicted_class = probabilities.argmax(dim=1).item()
    confidence = probabilities[0, predicted_class].item()  # Вероятность предсказанного класса
    
    # Преобразуем числовой класс в строковый
    pred_dict = {
        0: 'Bad',
        1: 'Neutral',
        2: 'Good'
    }
    predicted_label = pred_dict.get(predicted_class, "Unknown")
    
    return predicted_label, confidence
# Вызов функции
class_label, prob = l_predict_class(
    model_lstm,
    speech,
    preprocess_fn=data_preprocessing,
    padding=my_padding,
    seq_len=SEQ_LEN,
    vocabulary=vocab_to_int,
    device='cpu'
)
print("Предсказанный класс:", class_label, 'С вероятностью:', round(prob, 2))

Предсказанный класс: Neutral С вероятностью: 0.91
